# Importación de librerias de trabajo

In [2]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import numpy as np
import statsmodels.api as sm

from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, FunctionTransformer
from sklearn.pipeline import Pipeline

# Lectura del dataset Limpio

In [3]:
ruta = "C:/Users/Roland/Documents/UNAD_MEXICO_Matematicas_2022/Semestre_2024_1/ZZZ_Word_y_PT/DatosClientes/Clientes_Dos_Mil_Limpio.csv"
datasetT = pd.read_csv(ruta)

In [4]:
datasetT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 28 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Ocupacion           2000 non-null   object 
 1   NoPago              2000 non-null   int64  
 2   TotalMonto          2000 non-null   int64  
 3   EstadoContrato      2000 non-null   object 
 4   Producto            2000 non-null   object 
 5   FechaInicioC        2000 non-null   object 
 6   FechaAdjudicación   2000 non-null   object 
 7   FechaUltimoPago     2000 non-null   object 
 8   FechaProyectadaFin  2000 non-null   object 
 9   MontoVencido        2000 non-null   int64  
 10  Mensualidad         2000 non-null   int64  
 11  Ingresos            2000 non-null   int64  
 12  Legal               2000 non-null   object 
 13  EdadActual          2000 non-null   int64  
 14  AZULEstatus         2000 non-null   object 
 15  AdjudicaReal        2000 non-null   object 
 16  PagosP

# Ver la informacion leida del dataset

In [5]:
datasetT.head()


,Ocupacion,NoPago,TotalMonto,EstadoContrato,Producto,FechaInicioC,FechaAdjudicación,FechaUltimoPago,FechaProyectadaFin,MontoVencido,...,Genero,PersonaTipo,dtEstadoContrato,GrupoOrigenVenta,SemanasAdjud,Pagos9,Domicilia_Pago,Email,timetotV,Y
0,ejecutivo (a),35,302783,subastado con 013,autos,2015-04-17,2015-04-17,2018-02-21,2020-04-17,5046,...,mujer,fisica,"subasta (0,20]",evento,0.0,9,1,1,36,1
1,coordinador (a),38,211366,subastado con 012,autos,2015-02-20,2015-02-20,2018-02-08,2020-02-20,7044,...,hombre,fisica,"subasta (0,20]",modulo-agencia,0.0,9,1,1,40,1
2,arquitecto (a),50,323640,sorteo,autos,2015-02-20,2016-01-15,2019-02-18,2020-02-20,0,...,mujer,fisica,sorteo,modulo-agencia,47.0,8,1,1,50,1
3,gestor (a),60,377036,sorteo,autos,2015-04-17,2015-07-17,2020-03-05,2020-04-17,12566,...,mujer,fisica,sorteo,modulo-agencia,13.0,9,0,1,61,1
4,propietario (a),45,392349,subastado con 013,autos,2015-04-17,2015-04-17,2018-12-26,2020-04-17,6539,...,hombre,moral,"subasta (0,20]",modulo-agencia,0.0,9,0,1,46,1


# Ver estadística básica

In [6]:
datasetT.describe()

,NoPago,TotalMonto,MontoVencido,Mensualidad,Ingresos,EdadActual,PagosPuntuales,CostoAdmin,SemanasAdjud,Pagos9,Domicilia_Pago,Email,timetotV,Y
count,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,43.874000,281266.522000,10447.515500,4720.824500,23616.397000,38.624500,41.295000,19786.640500,41.630143,8.067000,0.168500,0.981000,45.569000,0.739500
std,13.788516,109728.153403,12004.923357,1771.529555,10306.101321,11.368954,14.954056,6841.694826,29.490978,1.024713,0.374403,0.136559,13.378742,0.439017
min,3.000000,12705.000000,0.000000,2000.000000,1440.000000,18.000000,0.000000,1571.000000,0.000000,1.000000,0.000000,0.000000,8.000000,0.000000
25%,34.000000,209467.000000,2421.000000,3491.000000,16358.750000,30.000000,30.000000,14772.500000,29.750000,8.000000,0.000000,1.000000,36.000000,0.000000
50%,47.000000,282068.500000,6783.500000,4701.000000,22905.500000,37.000000,44.500000,20630.000000,43.000000,8.000000,0.000000,1.000000,48.000000,1.000000
75%,56.000000,343684.750000,13512.000000,5727.500000,29489.250000,47.000000,54.000000,24845.250000,52.000000,9.000000,0.000000,1.000000,57.000000,1.000000
max,60.000000,842910.000000,94088.000000,14048.000000,72330.000000,82.000000,60.000000,39228.000000,296.000000,9.000000,1.000000,1.000000,68.000000,1.000000


# Transformar los datos del dataset de trabajo

# Dividir en conjunto de entrenamiento y prueba

In [7]:
df = pd.DataFrame(datasetT)

# Tomar el 80% de los datos de manera aleatoria para entrenamiento
df_entrenamiento = df.sample(frac=0.8, random_state=1)

# Usar el 20% restante para pruebas
df_prueba = df.drop(df_entrenamiento.index)

In [ ]:
df_entrenamiento['FechaInicioC'] = pd.to_datetime(df_entrenamiento['FechaInicioC']).astype(int) // 10**9
print(df_entrenamiento)

In [ ]:
data = {
    'Fecha': ['2024-11-01', '2023-05-23', '2022-01-15', '2025-07-30'],
    'Anio' : [0,0,0,0]
}
df = pd.DataFrame(data)

# Convertir la fecha de texto a un formato datetime y luego a número de timestamp (en segundos)
df['Anio'] = pd.to_datetime(df['Fecha']).view(int)

print(df)

In [ ]:
print("Entrenamiento (80%):")
print(df_entrenamiento)


In [17]:
df_entrenamiento.to_csv('C:/Users/Roland/Documents/UNAD_MEXICO_Matematicas_2022/Semestre_2024_1/ZZZ_Word_y_PT/DatosClientes/Clientes_Dos_Mil_TransEntrenamiento.csv', index=False, quoting=csv.QUOTE_NONNUMERIC)

In [ ]:
print("\nPruebas (20%):")
print(df_prueba)

In [12]:
df_prueba.to_csv('C:/Users/Roland/Documents/UNAD_MEXICO_Matematicas_2022/Semestre_2024_1/ZZZ_Word_y_PT/DatosClientes/Clientes_Dos_Mil_TransPrueba.csv', index=False, quoting=csv.QUOTE_NONNUMERIC)

# Realizar la transformacion de datos

In [ ]:

# Definir las columnas categóricas y numéricas
columnas_categoricas = ['Ocupacion', 'EstadoContrato', 'Producto', 'Legal', 'AZULEstatus',
           'AdjudicaReal', 'Genero', 'PersonaTipo', 'dtEstadoContrato', 'GrupoOrigenVenta']
columnas_numericas = ['NoPago', 'TotalMonto', 'MontoVencido', 'Mensualidad', 'Ingresos',
           'EdadActual', 'CostoAdmin', 'SemanasAdjud', 'Pagos9', 'Domicilia_Pago',
            'Email', 'timetotV']
# Crear los transformadores específicos
preprocesador = ColumnTransformer(
    transformers=[
        ('cat', OrdinalEncoder(), columnas_categoricas),
        ('num', StandardScaler(), columnas_numericas)
    ],
    remainder='passthrough'  # Mantener las columnas que no se transforman
)

# Aplicar la transformación
data_transformada = preprocesador.fit_transform(datasetT)
